# Code for YouTube Data API

I ended up not using the Data API because it only could give me total views and not minutes viewed as well. See the section on the YouTube Analytics API below

In [11]:
# Sample Python code for youtube.search.list
# See instructions for running these code samples locally:
# 

import os
import json
import requests
from time import sleep
import csv
import io
import datetime
from pathlib import Path
from github import Github

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]

def authenticate():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "../../../../client_secret_youtube_data_api_explorer.json"

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)
    return youtube


In [ ]:
youtube = authenticate()

In [22]:
request = youtube.videos().list(
        part="statistics",
        id="iH8b-5BxtuY,VT75azo1Ano,Q_8BOnQvI1Q"
)
response = request.execute()

print(json.dumps(response, indent = 2))

{
  "kind": "youtube#videoListResponse",
  "etag": "9oczuDzOHrj6KJflyfPV7e_VI8Y",
  "items": [
    {
      "kind": "youtube#video",
      "etag": "JA2HUsFYV6opccHE4cCylypBy3s",
      "id": "iH8b-5BxtuY",
      "statistics": {
        "viewCount": "49207",
        "likeCount": "264",
        "dislikeCount": "17",
        "favoriteCount": "0",
        "commentCount": "18"
      }
    },
    {
      "kind": "youtube#video",
      "etag": "_iPlGMTvQDRD3JDMBZTJ7OHcR5Y",
      "id": "VT75azo1Ano",
      "statistics": {
        "viewCount": "5454",
        "likeCount": "10",
        "dislikeCount": "0",
        "favoriteCount": "0",
        "commentCount": "1"
      }
    },
    {
      "kind": "youtube#video",
      "etag": "eMcGF8QSTBxsq0s46SFcSet1iOw",
      "id": "Q_8BOnQvI1Q",
      "statistics": {
        "viewCount": "4603",
        "likeCount": "35",
        "dislikeCount": "3",
        "favoriteCount": "0",
        "commentCount": "3"
      }
    }
  ],
  "pageInfo": {
    "totalResu

In [ ]:
request = youtube.search().list(
    part="snippet",
    channelId="UChJEp_twCCM7ZibBKJbl_Jg",
    channelType="any",
    maxResults=25,
    q="debussy",
    type="video",
    videoType="any"
)
response = request.execute()

print(json.dumps(response, indent = 2))


# Code for YouTube Analytics API

See sample code at https://developers.google.com/youtube/analytics/reference/reports/query#python


## Configuration

Run once at the start

In [79]:
import os
import json
import requests
from time import sleep
import csv
import io
import datetime
from pathlib import Path
from github import Github

import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

scopes = ['https://www.googleapis.com/auth/yt-analytics.readonly']

# Disable OAuthlib's HTTPs verification when running locally.
# *DO NOT* leave this option enabled when running in production.
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'

api_service_name = 'youtubeAnalytics'
api_version = 'v2'
client_secrets_file = '../../../../client_secret_youtube_analytics_download.json'
def get_service():
  flow = InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes)
  credentials = flow.run_console()
  return build(api_service_name, api_version, credentials = credentials)

def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()
  return response
    
# -----------------
# utility functions
# -----------------

def generate_utc_date():
    whole_time_string_z = datetime.datetime.utcnow().isoformat() # form: 2019-12-05T15:35:04.959311
    date_z = whole_time_string_z.split('T')[0] # form 2019-12-05
    return date_z

# read from a CSV file into a list of dictionaries (representing a table)
def read_dicts_from_csv(filename):
    with open(filename, 'r', newline='', encoding='utf-8') as file_object:
        dict_object = csv.DictReader(file_object)
        table = []
        for row in dict_object:
            table.append(row)
    return table

# write a list of dictionaries to a CSV file
def write_dicts_to_csv(table, filename, fieldnames):
    with open(filename, 'w', newline='', encoding='utf-8') as csv_file_object:
        writer = csv.DictWriter(csv_file_object, fieldnames=fieldnames)
        writer.writeheader()
        for row in table:
            writer.writerow(row)

# write a list of lists to a CSV file
def write_lists_to_csv(file_name, array):
    with open(file_name, 'w', newline='', encoding='utf-8') as file_object:
        writer_object = csv.writer(file_object)
        for row in array:
            writer_object.writerow(row)


The following line performs the authentication. Run it once at the start of the session. Not sure how long the session lasts but it could be something like 30 days?

In [ ]:
youtubeAnalytics = get_service()

## Script body

Can be run multiple times with the same session from previous cell. 

Note: the video filter can send up to 500 IDs. As of 2021-02-02, I have 268 videos I'm tracking, so at some point, this may have to be broken into two or more API calls.

In [103]:
# Load data about videos to monitor. 
video_metadata_filename = 'video-metadata.csv'
metadata = read_dicts_from_csv(video_metadata_filename)

filter_string = 'video=='
output_header_list = ['date']
for video in metadata:
#for video in metadata[0:5]:
    filter_string += video['id'].strip() + ','
    output_header_list.append(video['id'].strip())
# remove final trailing comma
filter_string = filter_string[:len(filter_string)-1]

todays_date_utc = generate_utc_date()


In [104]:
print('sending request to API')
result = execute_api_request(
    youtubeAnalytics.reports().query,
    ids='channel==MINE',
    startDate='2013-01-01', # don't have any videos dated earlier than that
    endDate=todays_date_utc,
    metrics='estimatedMinutesWatched,views',
    filters=filter_string,
    dimensions='video'
    )
#print(json.dumps(result, indent=2))
print('done retrieving data from API')

sending request to API
done retrieving data from API


In [106]:
# The table data will be in the form of a list of lists
api_data = result['rows']

minutes_table = [list(output_header_list)] # copy, not rename
views_table = [list(output_header_list)]

minutes_row = [todays_date_utc]
views_row = [todays_date_utc]
for header in output_header_list[1:]:
    found = False
    for video in api_data:
        if video[0] == header:
            found = True
            minutes_row.append(str(video[1]))
            views_row.append(str(video[2]))
            break
    if not found:
        minutes_row.append('')
        views_row.append('')
minutes_table.append(minutes_row)
views_table.append(views_row)

write_lists_to_csv('total_minutes_watched.csv', minutes_table)
write_lists_to_csv('total_views.csv', views_table)
print('done')

done


# Stop !!!

The following cells contain test queries that I used to debug and have kept for historical reference. They can be run stand-along after the authentication cell has been run.

In [62]:
result = execute_api_request(
    youtubeAnalytics.reports().query,
    ids='channel==MINE',
    startDate='2017-01-01',
    endDate='2017-12-31',
    metrics='estimatedMinutesWatched,views,averageViewDuration',
    dimensions='day'
    )
print(json.dumps(result, indent=2))

{
  "kind": "youtubeAnalytics#resultTable",
  "columnHeaders": [
    {
      "name": "day",
      "columnType": "DIMENSION",
      "dataType": "STRING"
    },
    {
      "name": "estimatedMinutesWatched",
      "columnType": "METRIC",
      "dataType": "INTEGER"
    },
    {
      "name": "views",
      "columnType": "METRIC",
      "dataType": "INTEGER"
    },
    {
      "name": "averageViewDuration",
      "columnType": "METRIC",
      "dataType": "INTEGER"
    }
  ],
  "rows": [
    [
      "2017-01-01",
      28,
      20,
      85
    ],
    [
      "2017-01-02",
      39,
      26,
      91
    ],
    [
      "2017-01-03",
      48,
      28,
      104
    ],
    [
      "2017-01-04",
      72,
      29,
      149
    ],
    [
      "2017-01-05",
      65,
      26,
      151
    ],
    [
      "2017-01-06",
      43,
      22,
      118
    ],
    [
      "2017-01-07",
      38,
      18,
      127
    ],
    [
      "2017-01-08",
      40,
      21,
      117
    ],
    [
   

In [63]:
result = execute_api_request(
    youtubeAnalytics.reports().query,
    ids='channel==MINE',
    startDate='2017-01-01',
    endDate='2021-02-03',
    metrics='estimatedMinutesWatched,views',
    filters='video==FAvnuY4HEks,iIgDMsXXQxg,VUSoRRlZFM8,f3NqLIXUtJs',
    dimensions='video'
    )
print(json.dumps(result, indent=2))

{
  "kind": "youtubeAnalytics#resultTable",
  "columnHeaders": [
    {
      "name": "video",
      "columnType": "DIMENSION",
      "dataType": "STRING"
    },
    {
      "name": "estimatedMinutesWatched",
      "columnType": "METRIC",
      "dataType": "INTEGER"
    },
    {
      "name": "views",
      "columnType": "METRIC",
      "dataType": "INTEGER"
    }
  ],
  "rows": [
    [
      "FAvnuY4HEks",
      74,
      33
    ],
    [
      "VUSoRRlZFM8",
      36,
      20
    ],
    [
      "f3NqLIXUtJs",
      77,
      20
    ],
    [
      "iIgDMsXXQxg",
      135,
      29
    ]
  ]
}
